# Part 2

In [1]:
import nltk
import sys
import os
from math import log
import numpy as np
from time import time
from gensim import corpora, models, similarities
import bs4
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

импортируем необходимое импортируемое, отдельно стоп-слова из nltk.download(), включим логирование

In [2]:
def preprocess(text):
    wnl = nltk.WordNetLemmatizer()
    text = bs4.BeautifulSoup(bs4.UnicodeDammit(text).unicode_markup, 'lxml').get_text()
    return [wnl.lemmatize(t) for t in text.lower().replace(',','').split()]

создаем функцию для лемматизации + токенизации + дехтмлизации текста

In [3]:
texts = []
for filename in os.listdir("/home/tigran/Roba/pyworks/modomawka/no4/"):
    with  open("/home/tigran/Roba/pyworks/modomawka/no4/" + filename, 'r', encoding = 'ISO-8859-1') as f: 
        print(filename)
        texts.append(preprocess(f.read()))

reut2-015.sgm
reut2-003.sgm
reut2-020.sgm
reut2-001.sgm
reut2-009.sgm
reut2-011.sgm
reut2-000.sgm
reut2-007.sgm
reut2-006.sgm
reut2-016.sgm
reut2-010.sgm
reut2-018.sgm
reut2-019.sgm
reut2-012.sgm
reut2-002.sgm
reut2-004.sgm
reut2-008.sgm
reut2-005.sgm
reut2-013.sgm
reut2-021.sgm
reut2-017.sgm
reut2-014.sgm


загрузим коллекцию обработанных текстов (названия файлов выводятся для наглядности (и для поиска багов, был битый текст))

In [4]:
dictionary = corpora.Dictionary(texts)
print('Original: {}'.format(dictionary))
dictionary.filter_extremes(no_below = 5, no_above = 0.5, keep_n=None)
dictionary.save('nips.dict')
print('Filtered: {}'.format(dictionary))

corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize('nips.mm', corpus) # store on disc


Original: Dictionary(148212 unique tokens: ['09:51:38.05', 'f5883reute', 'energy-based', 'mcghee', 'bc-warner-communications']...)
Filtered: Dictionary(11122 unique tokens: ['vulnerability', 'supply-demand', 'olivetti', '302', '13.50']...)


создадим словарь

In [8]:
def perplexity(model, corpus):
    corpus_length = 0
    log_likelihood = 0
    topic_profiles = model.state.get_lambda() / np.sum(model.state.get_lambda(), axis=1)[:, np.newaxis]
    for document in corpus:
        gamma, _ = model.inference([document])
        document_profile = gamma / np.sum(gamma)
        for term_id, term_count in document:
            corpus_length += term_count
            term_probability = np.dot(document_profile, topic_profiles[:, term_id])
            log_likelihood += term_count * log(term_probability)
    perplexity = np.exp(-log_likelihood / corpus_length)
    return perplexity

определим функцию перплексии

In [9]:
start = time()
model = models.ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=30, chunksize=50, update_every=1, passes=2)
print('Perplexity: {}'.format(perplexity(model, corpus)))
print('Evaluation time: {}'.format((time()-start) / 60))

Perplexity: 7074.609683705144
Evaluation time: 0.25847225189208983


посчитали модель на 30 топиках. хоть перплексити в абсолютном выражении и просто попугаи, но сдается мне, что их чересчур много. дальше будет показано, что модель и вправду очень плохая.

In [16]:
model.print_topic(6)


'0.004*--- + 0.004*apr + 0.002*cq + 0.001*gq + 0.001*twa + 0.001*0 + 0.001*mth + 0.001*comex + 0.001*louvre + 0.001*platinum'

дааа, судя по коэффициентам, что-то у модели не задалось...

In [7]:
for position in range(10):
    for topic in range(10):
        print(model.show_topic(topic)[position][1].center(12, ' ')) 
    print()


    ---     
    ---     
    apr     
    ---     
   magma    
    ---     
 jefferies  
    apr     
    ---     
    ---     

    apr     
    adb     
    ---     
    apr     
 fairchild  
    apr     
    apr     
    ---     
    apr     
    apr     

    g-7     
    csr     
     cq     
    csr     
interconnect
     cq     
    g-7     
    csr     
     cq     
   louvre   

   louvre   
    taft    
    twa     
borg-warner 
    frn     
   louvre   
   louvre   
     0      
     gq     
   caesar   

    twa     
     cq     
   dixons   
 jefferies  
   parker   
     gq     
    twa     
     gq     
  gillette  
    csr     

     0      
    apr     
   caesar   
   dixons   
  rudolph   
     0      
    0/92    
     cq     
  noranda   
    twa     

   eddie    
     ai     
     gq     
     0      
    taft    
    csr     
    ---     
    adb     
     0      
    g-7     

     cq     
   dixons   
    taft    
     cq     
    ---     
    unc     
    t

честное слово, не разобрался, как вывести слова красиво в шеренгу, но это и не поможет - модель получилась, мягко скажем, отвратная. сказывается и плохой препроцессинг (был сделан вслепую, кроме обработки хтмл), и куцый набор текстов (я находился в крайне сложном положении из-за необходимости загрузиться с арх линукса на внешнем диске, на котором не работали почему-то pacman и невозможно было поставить нормальный архиватор, работающий с .rar и .zip, поэтому выбор на репозитории UCI был крайне ограничен. плюс предлагаемые данные bag of words мне не удалось заставить работать с этим алгоритмом из-за отсутствия id2word ключей в словаре (там уже готовые словари и числовые пары слово-частота), и я не уверен, что это вообще возможно. 
Интерпретируя полученные результаты, можно сказать, что они интерпретации поддаются только в негативном ключе - смысла в них нет. Попробуем интереса ради переобучиться на более сложной модели

In [17]:
start = time()
model2 = models.ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=100, update_every=0, passes=40)
print('Evaluation time: {}'.format((time()-start) / 60))
print('Perplexity: {}'.format(perplexity(model2, corpus)))

Evaluation time: 4.752013158798218
Perplexity: 6156.594719094331


Как видно, модель считалась ощутимо дольше, и перплексия стала на тысячу попугаев меньше - однако, их все равно многовато. Оценим, глянув на топики

In [21]:
model2.print_topic(6)

"0.010*borg-warner + 0.007*texaco's + 0.006*gaf + 0.006*dome's + 0.006*uaw + 0.004*adb + 0.004*fmc + 0.003*crazy + 0.003*eddie + 0.003*reebok"

увы, сколько-нибудь более осмысленной модель не стала. слишком малы данные и плох препроцессинг для получения осмысленных результатов